### 正常训练

In [11]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
nursery = fetch_ucirepo(id=76) 
  
# data (as pandas dataframes) 
X = nursery.data.features 
y = nursery.data.targets

print(X.shape)
print(y.shape)

print(type(X))
print(type(y))
# metadata 
print(nursery.metadata) 
  
# variable information 
print(nursery.variables)


(12960, 8)
(12960, 1)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
{'uci_id': 76, 'name': 'Nursery', 'repository_url': 'https://archive.ics.uci.edu/dataset/76/nursery', 'data_url': 'https://archive.ics.uci.edu/static/public/76/data.csv', 'abstract': ' Nursery Database was derived from a hierarchical decision model originally developed to rank applications for nursery schools.', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 12960, 'num_features': 8, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1989, 'last_updated': 'Sun Jan 14 2024', 'dataset_doi': '10.24432/C5P88W', 'creators': ['Vladislav Rajkovic'], 'intro_paper': {'ID': 372, 'type': 'NATIVE', 'title': 'An application for admission in public school systems', 'authors': 'M. Olave, V. Rajkovic, M. Bohanec'

KeyboardInterrupt: 

### Gmean、mAUC、PPVs

In [2]:
from ucimlrepo import fetch_ucirepo # 加载数据集
from scipy.stats import gmean
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score,accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio  # 从.mat文件中读取数据集
################################################################加载数据集################################################
# 数据集
#mat_data = sio.loadmat('../data/dataset/Connect4.mat')
# 提取变量
#dataset_x = mat_data['X']
#dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]

  
# fetch dataset 
nursery = fetch_ucirepo(id=76) 
  
# data (as pandas dataframes) 
X = nursery.data.features 
y = nursery.data.targets 

dataset_x = X.values
dataset_y = y.values[:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
# One-hot encode target variable
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(dataset_y.reshape(-1, 1))
print(y_onehot.shape)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset_x, y_onehot, test_size=0.3, random_state=42, stratify=dataset_y)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Multi-Layer Perceptron (MLP) model
mlp = MLPClassifier(hidden_layer_sizes=(10), max_iter=800, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Predict probabilities for the test set
# 计算m-AUC
y_pred_proba = mlp.predict_proba(X_test_scaled)


# 计算accuracy
y_pred = mlp.predict(X_test_scaled)
# Convert one-hot encoded test labels back to single class labels
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)


# 计算每个类别的 Precision
class_precisions = precision_score(y_test_labels, y_pred_labels, average=None)


class_accuracies = {}
for class_label in np.unique(y_test_labels):
    indices = (y_test_labels == class_label)
    
    class_accuracies[class_label] = accuracy_score(y_test_labels[indices], y_pred_labels[indices])
# Compute AUC for each class
fpr = {}
tpr = {}
roc_auc = {}
for i in range(y_onehot.shape[1]):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
#print(y_pred_proba.shape)
#print("预测的概率：",y_pred_proba)
# Compute macro AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(y_onehot.shape[1])]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(y_onehot.shape[1]):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= y_onehot.shape[1]
macro_auc = auc(all_fpr, mean_tpr)

# Plot AUC curves
plt.figure(figsize=(8, 6))
for i in range(y_onehot.shape[1]):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
plt.plot(all_fpr, mean_tpr, label=f'Macro-average (AUC = {macro_auc:.2f})', linestyle='--', color='navy')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

macro_auc
class_accuracies
# 计算几何平均准确率
geometric_mean_accuracy_1 = np.prod(list(class_accuracies.values()))**(1 / len(class_accuracies))

geometric_mean_accuracy_2=gmean(list(class_accuracies.values()))
geometric_mean_accuracy_3=gmean(class_precisions)

print(macro_auc)
print(class_accuracies)
print(geometric_mean_accuracy_1)
print(geometric_mean_accuracy_2)
print(geometric_mean_accuracy_3)
print(class_precisions)

(12960, 5)


ValueError: could not convert string to float: 'usual'